In [1]:
# ==========================================
# ✅ CPU-friendly TensorFlow setup
# ==========================================

# Enable inline plotting in Jupyter notebook
%matplotlib inline

# ---- Environment setup ----
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"        # Force CPU only (ignore GPUs)
os.environ["KERAS_BACKEND"] = "tensorflow"       # Use TensorFlow backend
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "1"        # Enable optimized CPU kernels
os.environ["OMP_NUM_THREADS"] = "4"              # Limit parallel threads
os.environ["MKL_NUM_THREADS"] = "4"              # Keep CPU responsive

# ---- Core libraries ----
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Scikit-learn utilities ----
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder

# ---- Image utilities ----
import matplotlib.image as mpimg
from scipy import ndimage

# ---- TensorFlow / Keras ----
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, Dropout, Flatten, BatchNormalization,
    Conv2D, MaxPooling2D, Activation
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import TensorBoard

import pickle
import time  # for timestamping model runs

# ---- Ensure TensorFlow uses CPU only ----
for dev_type in ["GPU", "TPU", "MPS"]:
    try:
        tf.config.set_visible_devices([], dev_type)
    except Exception:
        pass

# ---- Optional: limit TensorFlow thread usage ----
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(2)
tf.keras.backend.set_floatx("float32")

# ---- Print environment info ----
print("===================================")
print("TensorFlow version:", tf.__version__)
print("Devices available:", tf.config.list_physical_devices())
print("GPU available:", len(tf.config.list_physical_devices('GPU')) > 0)
print("→ Running on CPU only 💻")
print("===================================")


TensorFlow version: 2.19.0
Devices available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
GPU available: False
→ Running on CPU only 💻


In [2]:
def unpickle(file):
    """
    Load and return the contents of a pickled CIFAR-10 batch file.
    
    Args:
        file (str): Path to the pickled CIFAR-10 batch file
        
    Returns:
        dict: Dictionary containing the CIFAR-10 batch data
    """
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load CIFAR-10 training and test datasets
training_data = unpickle('./cifar-10/data_batch_1')  # Load first training batch
test_data = unpickle('./cifar-10/test_batch')       # Load test batch

# Extract features (images) and labels from the loaded data
X_train = training_data.get(b'data')        # Training images (flattened)
train_labels = training_data.get(b'labels')  # Training labels

X_test = test_data.get(b'data')             # Test images (flattened)
test_labels = test_data.get(b'labels')       # Test labels

# Reshape the data into the correct format for TensorFlow/Keras
# Original shape: (num_images, 3072) where 3072 = 32*32*3 (flattened 32x32 RGB images)
# New shape: (num_images, 32, 32, 3) - 4D tensor with (batch, height, width, channels)
X_train = X_train.reshape((len(X_train), 3, 32, 32)).transpose(0, 2, 3, 1)
X_test = X_test.reshape((len(X_test), 3, 32, 32)).transpose(0, 2, 3, 1)

# Normalize pixel values to [0, 1] range for better training stability
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
# This transforms integer labels into a binary matrix where each row corresponds to one sample
# and each column corresponds to a class (10 classes for CIFAR-10)
y_train = to_categorical(train_labels, 10)  # 10 classes in CIFAR-10
y_test = to_categorical(test_labels, 10)    # 10 classes in CIFAR-10

# Print dataset information
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Test data shape:", X_test.shape)
print("Test labels shape:", y_test.shape)
print("Number of classes:", y_train.shape[1])

Training data shape: (10000, 32, 32, 3)
Training labels shape: (10000, 10)
Test data shape: (10000, 32, 32, 3)
Test labels shape: (10000, 10)
Number of classes: 10


In [3]:
# Create a unique name for the TensorBoard log directory with a timestamp
# This helps in tracking different training runs separately
model_name = 'CIFAR-10_CNN_Dropout_10_Percent_{}'.format(int(time.time()))

# Set up TensorBoard callback for logging training metrics
# log_dir: Directory where to save the log files to be parsed by TensorBoard
# histogram_freq: Frequency (in epochs) at which to compute activation histograms
# write_graph: Whether to visualize the graph in TensorBoard
# write_images: Whether to write model weights to visualize as image in TensorBoard
tensorboard = TensorBoard(
    log_dir='logs/{}'.format(model_name),
    histogram_freq=1,      # Log histograms every epoch
    write_graph=True,      # Visualize the computation graph
    write_images=True,     # Save model weights as images
    update_freq='epoch'    # Log metrics at the end of each epoch
)

print(f"TensorBoard logs will be saved to: logs/{model_name}")
print("To view the TensorBoard, run the next cell after training the model.")

TensorBoard logs will be saved to: logs/CIFAR-10_CNN_Dropout_10_Percent_1760622610
To view the TensorBoard, run the next cell after training the model.


In [4]:
# Neural network parameters
#-----------------------------------------------
#-----------------------------------------------
batch_size = 64 # 
epochs = 30 # 
dropoutRate = 0.1
#-----------------------------------------------
#-----------------------------------------------
num_classes = 10
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols,3)

# Model
model = Sequential()
#-----------------------------------------------
#-----------------------------------------------
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=input_shape)) 
#-----------------------------------------------
#-----------------------------------------------
model.add(MaxPooling2D(pool_size=(2, 2)))
#-----------------------------------------------
#-----------------------------------------------
model.add(Dropout(dropoutRate)) # Value between 0 and 1 
#-----------------------------------------------
model.add(BatchNormalization())
#-----------------------------------------------
model.add(Conv2D(32, (3, 3), activation='relu')) 
#-----------------------------------------------
#-----------------------------------------------
model.add(MaxPooling2D(pool_size=(2, 2)))
#-----------------------------------------------
#-----------------------------------------------
model.add(Dropout(dropoutRate)) # Value between 0 and 1 
#-----------------------------------------------
model.add(BatchNormalization())
#-----------------------------------------------
#-----------------------------------------------
model.add(Flatten()) 
model.add(Dense(50, activation='relu')) 
#-----------------------------------------------
#-----------------------------------------------
model.add(Dropout(dropoutRate)) # Value between 0 and 1 
#-----------------------------------------------
#-----------------------------------------------
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Trainen van het CNN
history = model.fit(X_train, y_train,batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1, callbacks = [tensorboard])

Epoch 1/30


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.2607 - loss: 2.1786 - val_accuracy: 0.1200 - val_loss: 2.7783
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.4591 - loss: 1.5268 - val_accuracy: 0.1370 - val_loss: 3.6643
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.5228 - loss: 1.3425 - val_accuracy: 0.1315 - val_loss: 3.5035
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5568 - loss: 1.2438 - val_accuracy: 0.3480 - val_loss: 1.9294
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5988 - loss: 1.1260 - val_accuracy: 0.4680 - val_loss: 1.5579
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6387 - loss: 1.0565 - val_accuracy: 0.4785 - val_loss: 1.5421
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6512 - loss: 0.9536 - val_accuracy: 0.5320 - val_loss: 1.3573
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6740 - loss: 0.9132 - val_accuracy: 0.566

In [5]:
# Second model


NAME = 'CIFAR-10_CNN_Dropout_30_Percent'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [6]:
# Neural network parameters
#-----------------------------------------------
#-----------------------------------------------
batch_size = 64 # 
epochs = 30 # 
dropoutRate = 0.3
#-----------------------------------------------
#-----------------------------------------------
num_classes = 10
img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols,3)

# Model
model = Sequential()
#-----------------------------------------------
#-----------------------------------------------
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=input_shape)) 
#-----------------------------------------------
#-----------------------------------------------
model.add(MaxPooling2D(pool_size=(2, 2)))
#-----------------------------------------------
#-----------------------------------------------
model.add(Dropout(dropoutRate)) # Value between 0 and 1 
#-----------------------------------------------
model.add(BatchNormalization())
#-----------------------------------------------
model.add(Conv2D(32, (3, 3), activation='relu')) 
#-----------------------------------------------
#-----------------------------------------------
model.add(MaxPooling2D(pool_size=(2, 2)))
#-----------------------------------------------
#-----------------------------------------------
model.add(Dropout(dropoutRate)) # Value between 0 and 1 
#-----------------------------------------------
model.add(BatchNormalization())
#-----------------------------------------------
#-----------------------------------------------
model.add(Flatten()) 
model.add(Dense(50, activation='relu')) 
#-----------------------------------------------
#-----------------------------------------------
model.add(Dropout(dropoutRate)) # Value between 0 and 1 
#-----------------------------------------------
#-----------------------------------------------
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Trainen van het CNN
history = model.fit(X_train, y_train,batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1, callbacks = [tensorboard])

Epoch 1/30


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.1866 - loss: 2.4736 - val_accuracy: 0.1225 - val_loss: 2.9405
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.3352 - loss: 1.8492 - val_accuracy: 0.1100 - val_loss: 3.5161
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4017 - loss: 1.6800 - val_accuracy: 0.2170 - val_loss: 2.8671
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4364 - loss: 1.5556 - val_accuracy: 0.3470 - val_loss: 1.9246
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4629 - loss: 1.4930 - val_accuracy: 0.3995 - val_loss: 1.7825
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4785 - loss: 1.4377 - val_accuracy: 0.3360 - val_loss: 1.9108
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.4976 - loss: 1.3946 - val_accuracy: 0.3320 - val_loss: 1.9064
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.5112 - loss: 1.3463 - val_accuracy: 0.530

In [7]:
%load_ext tensorboard

In [8]:
%tensorboard --logdir logs

For further reading on tensorboard:
https://www.tensorflow.org/tensorboard/get_started 